Создание документации и статический анализ
=======================================

Скачаем интересующий нас код
--------------------------------------------

In [0]:
!rm -rf multiwii-firmware
!git clone https://github.com/multiwii/multiwii-firmware

Применим стиль
----------------------

In [0]:
!apt install clang-format

In [0]:
!cp -r multiwii-firmware multiwii-firmware.backup

In [0]:
!echo "BasedOnStyle: LLVM" > .clang-format
!echo "IndentWidth: 4" >> .clang-format
#!echo "IndentWidth: 8" >> .clang-format
#!echo "AlignEscapedNewlines: DontAlign" >> .clang-format
#!echo "AlignEscapedNewlines: Right" >> .clang-format
#!echo "AllowShortIfStatementsOnASingleLine : false" >> .clang-format

Применим форматирование ко всем файлам `.cpp`.

In [0]:
!clang-format -i multiwii-firmware/*.cpp

Сравним полученный результат с исходным.

In [0]:
 !diff multiwii-firmware.backup/IMU.cpp multiwii-firmware/IMU.cpp | head -n 60

**ЗАДАНИЕ: Найдите хотя бы три различия**

**ЗАДАНИЕ: Измените стиль согласно заданию**



*  сделайте так, чтобы отступы блоков кода были равны 6 символов
* сделайте так, чтобы директивы препроцессора вообще не отступали от левого края (IndentPPDirectives)
* сделайте так, чтобы перед цепочкой инициализаторов не ставились пробелы (SpaceBeforeCtorInitializerColon)
* сделайте так, чтобы форматирование допускало **однострочные** короткие циклы for/while (AllowShortLoopsOnASingleLine)


см. [справку по clang](https://clang.llvm.org/docs/ClangFormatStyleOptions.html) ([ru](https://translate.google.com/translate?hl=ru&sl=en&u=https://clang.llvm.org/docs/ClangFormatStyleOptions.html&prev=search)) 

Разберём код при помощи Doxygen
-----------------------------------------------

Поставим Doxygen

In [0]:
!apt install doxygen

Создадим простой файл настроек для Doxygen,  в котором укажем выдавать нам только XML-вывод.

In [0]:
!echo "GENERATE_XML = yes" > Doxyfile
!echo "GENERATE_HTML = no" >> Doxyfile
!echo "GENERATE_LATEX = no" >> Doxyfile
!echo "INPUT = multiwii-firmware" >> Doxyfile
!echo "QUIET = YES" >> Doxyfile
!cat Doxyfile

Запустим обработку исходных файлов.

In [0]:
!doxygen Doxyfile 2>/dev/null

Проверим, что Doxygen сделал свою работу и теперь в папке XML находятся результаты анализа

In [0]:
#!ls xml/

#!tar -czvf xml.tar.gz xml
#from google.colab import files
#files.download("xml.tar.gz")

Собираем из набора XML файлов которые создал Doxygen -- один общий файл анализа кода.

In [0]:
import lxml.etree as ET
index = ET.parse('xml/index.xml')
for f in index.xpath('/doxygenindex/compound[@kind="file"]'):
  contents = ET.parse("xml/" + f.attrib['refid'] + ".xml").find('compounddef')
  f.append(contents)
#ET.to_string(index)
index.write('full_doxygen.xml', pretty_print=False)

In [0]:
from google.colab import files
files.download("full_doxygen.xml")

Орформатируем этот код
----------------------------------

Скачиваем стиль.

In [0]:
!wget https://raw.githubusercontent.com/nkapyrin/cad_course/master/lab_04/xelatex_document_template.xsl

Если папка для сборки документа уже существует, её нужно будет удалить чтобы скачать в неё конфигурацию для TeX.

In [0]:
!rm -rf tex
!wget https://github.com/nkapyrin/cad_course/raw/master/lab_04/tex.zip
!unzip tex.zip -d tex
#!ls tex

In [0]:
!xsltproc -o tex/document.tex xelatex_document_template.xsl full_doxygen.xml

In [0]:
!tar -czvf tex.tar.gz tex
from google.colab import files
files.download("tex.tar.gz")

Вот и всё
------------------------------------